In [40]:
import os
os.chdir('../')
os.getcwd()

'g:\\'

In [41]:
os.environ["MLFLOW_TRACKING_URI"] = 'https://dagshub.com/nikhil0035/Machine_Translation_using_Transformers.mlflow'
os.environ["MLFLOW_TRACKING_USERNAME"] = 'nikhil0035'
os.environ["MLFLOW_TRACKING_PASSWORD"] = '8d6d437e619e46ffbd80ee64bee271c7f105427c'

In [42]:
import warnings
from tqdm import tqdm
import os
from pathlib import Path

In [43]:
from src.Translate.components.dataset import BilingualDataset,causal_mask
from src.Translate.components.build_model import Prepare_model
from src.Translate.utils.common import *
from src.Translate.entity.config_entity import *

In [44]:
import torchtext.datasets as datasets
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import LambdaLR
import mlflow

from urllib.parse import urlparse
import torchmetrics


In [45]:
from src.Translate.config.configuration import ConfigurationManager


In [46]:
class Evaluation:
    def __init__(self,config:Config_Data,params:TrainingConfig,eval:EvaluationConfig) -> None:
        self.config = config
        self.params = params
        self.eval = eval
    
    @staticmethod
    def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
        sos_idx = tokenizer_tgt.token_to_id('[SOS]')
        eos_idx = tokenizer_tgt.token_to_id('[EOS]')

        # Precompute the encoder output and reuse it for every step
        encoder_output = model.encode(source, source_mask)
        # Initialize the decoder input with the sos token
        decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
        while True:
            if decoder_input.size(1) == max_len:
                break

            # build mask for target
            decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

            # calculate output
            out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

            # get next token
            prob = model.project(out[:, -1])
            _, next_word = torch.max(prob, dim=1)
            decoder_input = torch.cat(
                [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
            )

            if next_word == eos_idx:
                break

        return decoder_input.squeeze(0)
    
    def run_validation(self, model, validation_ds, tokenizer_src, tokenizer_tgt):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Using device:", device)
        self.model = model
        model = self.model.to(device)
        model.eval()
        count = 0
        max_len = self.config.seq_len
        source_texts = []
        expected = []
        predicted = []


        with torch.no_grad():
            batch_iterator = tqdm(validation_ds)
            for batch in batch_iterator:
                count += 1
                encoder_input = batch["encoder_input"].to(device) # (b, seq_len)
                encoder_mask = batch["encoder_mask"].to(device) # (b, 1, 1, seq_len)

                # check that the batch size is 1
                assert encoder_input.size(
                    0) == 1, "Batch size must be 1 for validation"

                model_out = self.greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

                source_text = batch["src_text"][0]
                target_text = batch["tgt_text"][0]
                model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

                source_texts.append(source_text)
                expected.append(target_text)
                predicted.append(model_out_text)

                if count ==5:
                    break

        metric = torchmetrics.CharErrorRate()
        self.cer = metric(predicted, expected)
       

        # Compute the word error rate
        metric = torchmetrics.WordErrorRate()
        self.wer = metric(predicted, expected)
      

        # Compute the BLEU metric
        metric = torchmetrics.BLEUScore()
        self.bleu = metric(predicted, expected)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.eval.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.eval.all_params)

            mlflow.log_metrics(
                {"CER": self.cer, "WER": self.wer , "Bleu":self.bleu}
            )

            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.pytorch.log_model(self.model, "model", registered_model_name="NMT_Transformers")
            else:
                mlflow.pytorch.log_model(self.model, "model")

In [47]:
from Translate.pipeline.stage_01_data_injestion import DataIngestionTrainingPipeline
data_obj = DataIngestionTrainingPipeline()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = data_obj.main()

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'

In [ ]:
from Translate.pipeline.stage_02_prepare_model import PrepareModelPipeline
prepare_model_obj = PrepareModelPipeline()
model = prepare_model_obj.main(tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size())


[2024-02-10 13:04:17,976: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-10 13:04:17,978: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-10 13:04:17,979: INFO: common: created directory at: artifacts]


In [ ]:
model_filename = latest_weights_file_path(config_obj)
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

[2024-02-10 13:05:15,626: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-10 13:05:15,629: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-10 13:05:15,631: INFO: common: created directory at: artifacts]


In [ ]:
try:
    config_instance = ConfigurationManager()
    config_obj = config_instance.get_config()
    param_obj = config_instance.get_training_config()
    eval_obj = config_instance.get_evaluation_config()

   

except Exception as e:
   raise e

In [32]:
Evaluation_obj =Evaluation(config=config_obj,params=param_obj,eval=eval_obj)

In [33]:
Evaluation_obj.run_validation(model,val_dataloader, tokenizer_src, tokenizer_tgt)

Using device: cuda


  0%|          | 4/3234 [00:35<7:51:28,  8.76s/it]


In [34]:
Evaluation_obj.log_into_mlflow()

2024/02/10 13:33:25 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/02/10 13:33:41 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
c:\Users\nikhi\anaconda3\envs\NMT\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'NMT_Transformers'.
2024/02/10 1